In [7]:
import pickle
import pandas as pd
import os

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
# from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
import rnn_classes
from raceDB import build_dataset, build_pred_dataset
import importlib
import datetime
from model_saver import model_saver, model_saver_wandb
import training_testing_gru
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, pad_packed_sequence,pad_sequence, unpack_sequence, unpad_sequence

In [8]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [9]:
torch.set_default_tensor_type(torch.FloatTensor)

In [10]:
def custom_l2(output, target):
    loss = torch.mean(abs(output-target), dim=1)
    return loss

In [11]:
def validation_CLE(x,y):
    loss_t = -torch.log(torch.exp(x)/torch.sum(torch.exp(x), dim=-1, keepdim=True))*y
    return loss_t

In [12]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
hidden_size = 32
states = ["NSW"]
data_file = 'gru_inputs_simple_normed_v7.fth'
raceDB = build_dataset(data_file, hidden_size ,state_filter=states,track_filter='Dapto', margin_type='boosted_sftmin',v6=True)
raceDB.create_new_weights_v2()
#raceDB.adjust_weights({"Dapto":10, "Gunnedah":10})

[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'last_start_price']]
(1246719, 23)
24
Latest date = 2023-05-20 00:00:00
size after state filter (309612, 25)
size after track filter (13416, 26)
(13416, 26)
Latest date = 2023-05-18 00:00:00
num_features_per_dog=24


  0%|          | 0/2571 [00:00<?, ?it/s]c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:
  0%|          | 0/1886 [00:00<?, ?it/s]c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:
100%|██████████| 1886/1886 [00:06<00:00, 311.04it/s]


number of races = 1886, number of unique dogs = 2572
0        (Dapto, 1.2361204114016693)
1       (Dapto, -1.3675934546785284)
2        (Dapto, 1.2361204114016693)
3        (Dapto, 1.2361204114016693)
4        (Dapto, 1.2361204114016693)
                    ...             
1881     (Dapto, 1.2361204114016693)
1882     (Dapto, 1.2361204114016693)
1883     (Dapto, 1.2361204114016693)
1884    (Dapto, -1.3675934546785284)
1885     (Dapto, 1.2361204114016693)
Length: 1886, dtype: object


In [13]:
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
raceDB.create_test_split_date(date)

Train examples 1443, Test examples 443


In [14]:
raceDB.create_dogs_test_split_date()

100%|██████████| 2571/2571 [00:00<00:00, 8477.44it/s]


In [15]:
raceDB.attach_races_to_dog_inputv2()

100%|██████████| 2572/2572 [00:00<00:00, 14119.04it/s]


In [16]:
raceDB.get_dog_test([1,2])[0][0].stats

tensor([-1.0574,  1.0000,  0.0000,  0.0000,  0.2435, -0.2883, -0.4453, -0.9133,
        -0.0038,  0.5145,  1.0728,  0.5096,  0.1498,  0.4129,  0.0000,  0.6402,
        -1.2826,  2.5025,  0.7069, -0.2590,  0.3186, -0.3829,  0.4385,  2.7991],
       device='cuda:0', dtype=torch.float64)

In [17]:
raceDB.create_batches(batch_days=1000)

[(datetime.date(2019, 12, 1), datetime.date(2022, 7, 31))]
2022-07-31
Train examples [1443]


100%|██████████| 1/1 [00:00<00:00,  2.20it/s]


Train examples [2086]
Train examples [2086]
Train examples [1443]
Train examples [1443]


In [34]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None, prev_model_file=None, prev_model_version='450'):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="NEW GRU", config=config):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss_val", summary="min")
      wandb.define_metric("accuracy", summary="max")
      wandb.define_metric("ROI < 30", summary="max")
      wandb.define_metric("relu roi", summary="max")
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      # input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy
      print(config.input_type)
      if config.input_type == 'full':
        print('here1')
        input_size = raceDB.batches['packed_x'][0].data[0].shape[0]
      elif config.input_type == 'basic':
        print('here')
        raceDB.batches['packed_x'] = raceDB.batches['packed_x_basic']
        raceDB.batches['packed_y'] = raceDB.batches['packed_y_basic']
        input_size = raceDB.batches['packed_x'][0].data[0].shape[0]

      print(f"{input_size=}")

      


      raceDB.reset_hidden(num_layers=config['num_layers'], hidden_size=config['hidden_size'])
      model = rnn_classes.GRUNetv3_LN(input_size,config['hidden_size'], num_layers=config['num_layers'],fc0_size=config['f0_layer_size'], fc1_size=config['f1_layer_size'])

      if model_state_dict:
        model.load_state_dict(model_state_dict)
      if prev_model_file!=None:
        model_name = prev_model_file
        model_loc = f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_{prev_model_version}.pt"
        model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
        model.load_state_dict(model_data['model_state_dict'])
        config['parent model'] = prev_model_file
        model = model.to(device)
        optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
        optimizer.load_state_dict(model_data['optim'])
        # optimizer.to(device)
      else:
        optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])

      raceDB.to_cuda()

      criterion = nn.CrossEntropyLoss(reduction='none')

      model = model.to(device)

      # optimizer = optimizer.to(device)
      print(model)

      # and use them to train the model
      try:
        # train_double_loss_regular(model, dataset, criterion, optimizer, scheduler, config, crit2=custom_l2)
        training_testing_gru.train_regular_v3(model, dataset, criterion, optimizer, 'na', config)
      except (KeyboardInterrupt) as error:
        print(error)
        print("finished Early")
        
      # dataset.create_hidden_states_dict()
      raceDB.create_hidden_states_dict_v2()
      model_saver_wandb(model, optimizer, 450, 0.1, raceDB.hidden_states_dict_gru_v6,raceDB.train_hidden_dict , model_name="long nsw new  22000 RUN")
      if sweep:
        raceDB.reset_hidden()
    


    # and test its final performance
    #test(model, test_loader)

    return (model,dataset, optimizer)

In [19]:
len(raceDB.raceIDs)
wandb_config_static = {'hidden_size':128,
                       'stats':raceDB.stats_cols,
                       'races':states,
                       'datafile':data_file,
                       'latest_date':raceDB.latest_date,
                       'input_type':'basic',
                       'num_layers':2,
                       'batch_size': 750,
                       'dropout': 0.3,
                       'epochs': 10_000,
                       'learning_rate': 0.00002,
                       'optimizer': 'adamW',
                       'f0_layer_size':1024,
                       'f1_layer_size':128}

In [25]:
importlib.reload(training_testing_gru)
importlib.reload(rnn_classes)

C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA


<module 'rnn_classes' from 'c:\\Users\\Nick\\Documents\\GitHub\\grvmodel\\Python\\pytorch\\New Model\\LSTM\\rnn_classes.py'>

In [21]:
raceDB.reset_hidden(num_layers=wandb_config_static['num_layers'], hidden_size=wandb_config_static['hidden_size'])

In [22]:
torch.cuda.empty_cache()

In [23]:
for race in raceDB.racesDict.values():
    for dog in race.dogs:
        try:
            del dog.stats
        except:
            pass

torch.cuda.empty_cache()

In [33]:
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


{'batch_size': 10, 'dropout': 0.3, 'epochs': 300, 'f0_layer_size': 128, 'f1_layer_size': 64, 'hidden_size': 128, 'input_type': 'basic', 'learning_rate': 0.00191054804749302, 'len_data': 1886, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'races': ['NSW'], 'stats': "[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'last_start_price']]", 'validation_split': 0.1, 'datafile': 'gru_inputs_simple_normed_v7.fth', 'latest_date': '2023-05-18T00:00:00'}
300
{'batch_size': 10, 'dropout': 0.3, 'epochs': 300, 'f0_layer_size': 128, 'f1_layer_size': 64, 'hidden_size': 128, 'input_type': 'basic', 'learning_rate': 0.00191054804749302, 'len_data': 1886, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'races': ['NSW'], 'stats': "[['dist_last__1', 'box_last__1', 's

  0%|          | 0/300 [00:00<?, ?it/s]

[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmp_24i9d6hwandb-media\\8lzsagw5.table.json'


  7%|▋         | 20/300 [00:58<13:50,  2.97s/it]

...

[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmp_24i9d6hwandb-media\\hzkjjsog.table.json'


 93%|█████████▎| 280/300 [07:48<00:30,  1.52s/it]

[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmp_24i9d6hwandb-media\\c3jw6881.table.json'


100%|██████████| 300/300 [08:18<00:00,  1.66s/it]


ROI < 30,▂▁▄▇▇█▅█▆▅▆▆▄▆▅
ROI < 30 2,▃▁▅█▇█▅▇▅▄▅▆▅▆▅
accuracy,▁▆█▆▆▇▆█▇▆▆▇▆▆▆
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
flat_simple,▃▁▅▇▆█▆█▆▆▆▇▆▅▄
loss_1,██▇▇▆▅▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_val,▂▁▂▅▆▆▅▆▆▆▆▇▇▇█
multibet outlay < 30,▁▂▅▇██▇████████
multibet profit,▃▁▅█▇█▆▆▅▅▅▅▃▄▃
multibet profit < 30,▂▁▃▆▇█▅▇▅▄▅▆▄▅▄
multibet profit < 30 sd,▁▂▄▇██▇▇▇▇▇▇▆▇▆


In [28]:
sweep_config = {"method": "random"}

metric = {"name": "ROI < 30", "goal": "maximize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"value": "adamW"},
    "f0_layer_size": {"values": [256,128,64,32]},
    "f1_layer_size": {"values": [64,32]},
    "dropout": {"values": [0.3]},
    "input_type": {"values": ['full','basic']},
    "num_layers": {"values": [2,3,4]},
    'hidden_size':{'values':[32,64,128]},
    "len_data": {"value": len(raceDB.raceIDs)},
    "stats":{"value": raceDB.stats_cols},
    "races":{"value": states},
    "batch_size":{"value": 10},
    # "hidden_size": {"value":hidden_size}
}

sweep_config["parameters"] = parameters_dict

parameters_dict.update(
    {
        "epochs": {"values": [300]},
        "validation_split": {"value": 0.1},
        "loss": {
            "values": [ "CEL"],
            # "values": ["Huber", "MSE", "L1", "BCE", "Custom", "KL"]
            # 'value': 'l1_custom'
        },
        # "num_layers": {"values": [2]},
    }
)

parameters_dict.update(
    {
        "learning_rate": {
            # a flat distribution between 0 and 0.1
            "distribution": "uniform",
            "min": 0,
            "max": 0.002,
        },
    }
)

import pprint

pprint.pprint(sweep_config)


sweep_config

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'ROI < 30'},
 'parameters': {'batch_size': {'value': 10},
                'dropout': {'values': [0.3]},
                'epochs': {'values': [300]},
                'f0_layer_size': {'values': [256, 128, 64, 32]},
                'f1_layer_size': {'values': [64, 32]},
                'hidden_size': {'values': [32, 64, 128]},
                'input_type': {'values': ['full', 'basic']},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.002,
                                  'min': 0},
                'len_data': {'value': 1886},
                'loss': {'values': ['CEL']},
                'num_layers': {'values': [2, 3, 4]},
                'optimizer': {'value': 'adamW'},
                'races': {'value': ['NSW']},
                'stats': {'value': "[['dist_last__1', 'box_last__1', "
                                   "'speed_avg_1', 'split_speed_avg_1', "
               

{'method': 'random',
 'metric': {'name': 'ROI < 30', 'goal': 'maximize'},
 'parameters': {'optimizer': {'value': 'adamW'},
  'f0_layer_size': {'values': [256, 128, 64, 32]},
  'f1_layer_size': {'values': [64, 32]},
  'dropout': {'values': [0.3]},
  'input_type': {'values': ['full', 'basic']},
  'num_layers': {'values': [2, 3, 4]},
  'hidden_size': {'values': [32, 64, 128]},
  'len_data': {'value': 1886},
  'stats': {'value': "[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'last_start_price']]"},
  'races': {'value': ['NSW']},
  'batch_size': {'value': 10},
  'epochs': {'values': [300]},
  'validation_split': {'value': 0.1},
  'loss': {'values': ['CEL']},
  'learning_rate': {'distribution': 'uniform', 'min': 0, 'max': 0.002}}}

In [35]:
sweep_id = wandb.sweep(sweep_config, project="NEW GRU-sweeps")
CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=100)

Create sweep with ID: vnvru3zt
Sweep URL: https://wandb.ai/nickojelly/NEW%20GRU-sweeps/sweeps/vnvru3zt


wandb: Agent Starting Run: nhm5bayn with config:
wandb: 	batch_size: 10
wandb: 	dropout: 0.3
wandb: 	epochs: 300
wandb: 	f0_layer_size: 64
wandb: 	f1_layer_size: 32
wandb: 	hidden_size: 32
wandb: 	input_type: basic
wandb: 	learning_rate: 0.00034301030691257075
wandb: 	len_data: 1886
wandb: 	loss: CEL
wandb: 	num_layers: 2
wandb: 	optimizer: adamW
wandb: 	races: ['NSW']
wandb: 	stats: [['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'last_start_price']]
wandb: 	validation_split: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


{'batch_size': 10, 'dropout': 0.3, 'epochs': 300, 'f0_layer_size': 64, 'f1_layer_size': 32, 'hidden_size': 32, 'input_type': 'basic', 'learning_rate': 0.00034301030691257075, 'len_data': 1886, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'races': ['NSW'], 'stats': "[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'last_start_price']]", 'validation_split': 0.1}
300
{'batch_size': 10, 'dropout': 0.3, 'epochs': 300, 'f0_layer_size': 64, 'f1_layer_size': 32, 'hidden_size': 32, 'input_type': 'basic', 'learning_rate': 0.00034301030691257075, 'len_data': 1886, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'races': ['NSW'], 'stats': "[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'RunHomeTim

  0%|          | 0/300 [00:00<?, ?it/s]

[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmp_24i9d6hwandb-media\\snmtkl39.table.json'


...

 67%|██████▋   | 201/300 [10:07<02:51,  1.73s/it]

[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmp_24i9d6hwandb-media\\mf4ci46d.table.json'


 73%|███████▎  | 220/300 [11:06<03:38,  2.73s/it]

[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmp_24i9d6hwandb-media\\2k1jfsi0.table.json'


 79%|███████▊  | 236/300 [12:02<02:57,  2.77s/it]